In [1]:
import pandas as pd
import numpy as np
import psycopg2

conn  = psycopg2.connect(host="localhost", user="root", port=5432, database="W9sV6cL2dX", password="E5rG7tY3fH")





# define your query
query = """
        SELECT  
    fc.user_id, 
    fc.item_id, 
    fc.session_id, 
    to_timestamp(fc.evnt_stamp) as cur_evt_unix, 
    fc.type, 
    fc.strategy, 
    i.type,
    u.age,
    u.gender,
    u.country,
    CONCAT(itx.topics, iti.topic) AS Topics,
    to_timestamp(LEAD(evnt_stamp) OVER (
            PARTITION BY session_id
            ORDER BY
              evnt_stamp
          )) as next_evt_unix,
    ( (LEAD(evnt_stamp) OVER (
            PARTITION BY session_id
            ORDER BY
              evnt_stamp
          ) - fc.evnt_stamp)) as Timespent	  
    FROM 
        public.fct_hourly_metric fc
    Inner Join
        users u on u.id = fc.user_id
    left JOIN 
        items i ON fc.item_id = i.item_key
    LEFT JOIN 
        item_txt_content itx ON i.item_key = itx.item_key
    LEFT JOIN 
        item_img_content iti ON i.item_key = iti.item_key
    order by cur_evt_unix, fc.type  desc
        """

# execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# make sure to close the connection
conn.close()

# Now your data is in a pandas DataFrame, which you can check by:
print(df.head())

/tmp/ipykernel_3933575/928396368.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


   user_id                               item_id  \
0    21507  0aa69c75-fe3d-4fa4-b7b6-be5ddb84dc97   
1    21507  e51065ff-bf8e-481e-88a9-a73fe0e97bbb   
2    21507  cf669ac8-3467-454c-ae6e-d080ac085b4b   
3    21507  0aa69c75-fe3d-4fa4-b7b6-be5ddb84dc97   
4     2125  fbf59f88-29a6-4877-9c8e-efc7619c9788   

                             session_id              cur_evt_unix  type  \
0  f8e41567-0858-4376-ac68-b0701ee23285 2023-07-25 15:38:16+00:00  reco   
1  f8e41567-0858-4376-ac68-b0701ee23285 2023-07-25 15:38:21+00:00  reco   
2  f8e41567-0858-4376-ac68-b0701ee23285 2023-07-25 15:38:23+00:00  reco   
3  f8e41567-0858-4376-ac68-b0701ee23285 2023-07-25 15:38:23+00:00   evt   
4  8a9a9b2f-ed28-495d-b344-4d5dd7c7cd3b 2023-07-25 15:38:24+00:00  reco   

  strategy type   age gender  country                                 topics  \
0     None  txt  10.0   male  AdaLove         ["rugby ball", "cat", "robot"]   
1     None  txt  10.0   male  AdaLove  ["rugby ball", "ship", "soccer ball"]

In [2]:


#df to csv
df.to_csv('twotowers_source_data.csv', index=False)

In [3]:
df1= df[['user_id','item_id','session_id','age','gender','topics','country','timespent']]
#df1 to csv
df1.to_csv('df1.csv', index=False)
df1.head()

,user_id,item_id,session_id,age,gender,topics,country,timespent
0,21507,0aa69c75-fe3d-4fa4-b7b6-be5ddb84dc97,f8e41567-0858-4376-ac68-b0701ee23285,10.0,male,"[""rugby ball"", ""cat"", ""robot""]",AdaLove,5.0
1,21507,e51065ff-bf8e-481e-88a9-a73fe0e97bbb,f8e41567-0858-4376-ac68-b0701ee23285,10.0,male,"[""rugby ball"", ""ship"", ""soccer ball""]",AdaLove,2.0
2,21507,cf669ac8-3467-454c-ae6e-d080ac085b4b,f8e41567-0858-4376-ac68-b0701ee23285,10.0,male,rugby ball,AdaLove,NaN
3,21507,0aa69c75-fe3d-4fa4-b7b6-be5ddb84dc97,f8e41567-0858-4376-ac68-b0701ee23285,10.0,male,"[""rugby ball"", ""cat"", ""robot""]",AdaLove,0.0
4,2125,fbf59f88-29a6-4877-9c8e-efc7619c9788,8a9a9b2f-ed28-495d-b344-4d5dd7c7cd3b,NaN,unkwn,"[""basketball ball"", ""corn"", ""apple""]",unkwn,0.0


In [11]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import json
import ast

# Load your data
data = pd.read_csv('df1.csv')


def parse_topics(topics_str):
    # If the topics are a string representation of a list
    if isinstance(topics_str, str) and topics_str.startswith('[') and topics_str.endswith(']'):
        # Strip off the square brackets and split on commas
        return topics_str[1:-1].replace('"', '').replace("'", "").split(',')
    # If the topics are just a single string
    elif isinstance(topics_str, str):
        return [topics_str]
    else:
        # If topics_str is not a string (e.g., it's NaN)
        return []

# Apply the function to the 'topics' column
data['topics'] = data['topics'].apply(parse_topics)


# Define columns to be encoded
columns_to_encode = ['gender', 'country']

# Dictionary to save the label encoders for each column
label_encoders = {}

# Label encode 'gender' and 'country' columns
for col in columns_to_encode:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Get list of all unique topics
unique_topics = set()
for topics in data['topics']:
    for topic in topics:
        unique_topics.add(topic)

# Dictionary to map topics to integers
topic_to_int = {topic: i for i, topic in enumerate(unique_topics)}

# Replace topics with their corresponding integers
data['topics'] = data['topics'].apply(lambda x: [topic_to_int[topic] for topic in x])

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Separate features and labels for training set
train_features = [train_data[column].values for column in columns_to_encode]
train_features.append(np.stack(train_data['topics'].values))
train_labels = train_data['timespent'].values

# Separate features and labels for testing set
test_features = [test_data[column].values for column in columns_to_encode]
test_features.append(np.stack(test_data['topics'].values))
test_labels = test_data['timespent'].values

import tensorflow as tf

# Define the input layers
inputs = [tf.keras.layers.Input(shape=(1,), name=f'{col}_input') for col in columns_to_encode]
inputs.append(tf.keras.layers.Input(shape=(None,), name='topics_input'))  # The topics input can have arbitrary length

# Define the embedding layers
embeddings = [tf.keras.layers.Embedding(input_dim=len(label_encoders[col].classes_), output_dim=10)(input_layer) for col, input_layer in zip(columns_to_encode, inputs)]
embeddings.append(tf.keras.layers.Embedding(input_dim=len(unique_topics), output_dim=10)(inputs[-1]))

# Flatten the embeddings
flattened_embeddings = [tf.keras.layers.Flatten()(embedding) for embedding in embeddings]

# Concatenate the flattened embeddings
x = tf.keras.layers.Concatenate()(flattened_embeddings)

# Add dense layers
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Define the model
model = tf.keras.models.Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(train_features, train_labels, epochs=10, batch_size=32, validation_data=(test_features, test_labels))


ValueError: all input arrays must have the same shape

prev version 1 V

In [ ]:
# # Import the necessary libraries
# # Import the necessary libraries
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, MultiLabelBinarizer
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import json

# # Load your data
# data = pd.read_csv('df1.csv')
# # Define columns to be encoded
# columns_to_encode = ['gender', 'country', 'content_type']

# # Dictionary to save the label encoders for each column
# label_encoders = {}

# # Label encode 'gender', 'country', and 'content_type' columns
# for col in columns_to_encode:
#     le = LabelEncoder()
#     data[col] = le.fit_transform(data[col])
#     label_encoders[col] = le

# # Ensure topics are integer encoded. Here, we will encode each unique topic to a unique integer.
# mlb = MultiLabelBinarizer()
# topics_encoded = mlb.fit_transform(data['topics'])
# # Create a dictionary to map topics to integers
# topic_to_int = {topic: i for i, topic in enumerate(mlb.classes_)}
# # Replace topics with their corresponding integers
# data['topics'] = data['topics'].apply(lambda x: [topic_to_int[topic] for topic in x])

# # Split data into training and testing sets
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# # Separate features and labels for training set
# train_features = [train_data[column].values for column in columns_to_encode]
# train_features.append(np.stack(train_data['topics'].values))
# train_labels = train_data['timespent'].values

# # Separate features and labels for testing set
# test_features = [test_data[column].values for column in columns_to_encode]
# test_features.append(np.stack(test_data['topics'].values))
# test_labels = test_data['timespent'].values


,user_id,item_id,session_id,age,topics,timespent,gender_female,gender_male,gender_other,gender_unkwn,...,grapes,guitar,horse,mushroom,robot,rugby ball,ship,soccer ball,tennis racket,truck
0,21507,0aa69c75-fe3d-4fa4-b7b6-be5ddb84dc97,f8e41567-0858-4376-ac68-b0701ee23285,0.094737,"[rugby ball, cat, robot]",0.294118,0.0,1.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
1,21507,e51065ff-bf8e-481e-88a9-a73fe0e97bbb,f8e41567-0858-4376-ac68-b0701ee23285,0.094737,"[rugby ball, ship, soccer ball]",0.117647,0.0,1.0,0.0,0.0,...,0,0,0,0,0,1,1,1,0,0
2,21507,cf669ac8-3467-454c-ae6e-d080ac085b4b,f8e41567-0858-4376-ac68-b0701ee23285,0.094737,[rugby ball],0.000000,0.0,1.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
3,21507,0aa69c75-fe3d-4fa4-b7b6-be5ddb84dc97,f8e41567-0858-4376-ac68-b0701ee23285,0.094737,"[rugby ball, cat, robot]",0.000000,0.0,1.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
4,2125,fbf59f88-29a6-4877-9c8e-efc7619c9788,8a9a9b2f-ed28-495d-b344-4d5dd7c7cd3b,0.336842,"[basketball ball, corn, apple]",0.000000,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270261,32933,878a12f5-b19f-4c2e-a450-ee460e51fe0b,db14c1ed-e0ac-43ee-aff3-ed8b2fd82004,0.526316,"[brocolli, ship, rugby ball]",0.000000,0.0,1.0,0.0,0.0,...,0,0,0,0,0,1,1,0,0,0
270262,9552,49000889-bbe4-4a75-a65a-ba362104ed9f,6c321b38-84e5-41f2-931e-87697f9fd1b6,0.473684,"[truck, tennis racket, ship]",0.000000,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,1,0,1,1
270263,25962,39525912-af6a-4f04-b4cb-c6533e5bddf7,b7dd68d9-9c64-4927-9ddb-c500ac1c29d0,0.400000,[],0.000000,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
270264,25962,c4a2de90-e781-4421-9107-14bda0441140,b7dd68d9-9c64-4927-9ddb-c500ac1c29d0,0.400000,[robot],0.000000,1.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler




# Encode categorical columns to convert them into numerical
# You might want to save these encoders for later when you'll want to make predictions on new data
label_encoders = {}
for col in ['user_id', 'item_id', 'gender', 'country', 'type', 'Topics']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le


# Split your data into training and testing sets
train, test = train_test_split(data, test_size=0.2, random_state=42)

# Define the architecture of the User Tower
user_input = Input(shape=(1,), dtype=tf.int32, name='user_input')
user_embedding = Embedding(input_dim=data['user_id'].nunique(), output_dim=50, name='user_embedding')(user_input)
user_embedding = Flatten()(user_embedding)

# Add the other user features
user_age_input = Input(shape=(1,), dtype=tf.float32, name='user_age_input')
user_gender_input = Input(shape=(1,), dtype=tf.int32, name='user_gender_input')
user_country_input = Input(shape=(1,), dtype=tf.int32, name='user_country_input')

user_features = Concatenate()([user_embedding, user_age_input, user_gender_input, user_country_input])

user_tower = Dense(128, activation='relu')(user_features)
user_tower = Dense(64, activation='relu')(user_tower)

# Define the architecture of the Item Tower
item_input = Input(shape=(1,), dtype=tf.int32, name='item_input')
item_embedding = Embedding(input_dim=data['item_id'].nunique(), output_dim=50, name='item_embedding')(item_input)
item_embedding = Flatten()(item_embedding)

# Add the other item features
item_type_input = Input(shape=(1,), dtype=tf.int32, name='item_type_input')
item_topics_input = Input(shape=(1,), dtype=tf.int32, name='item_topics_input')

item_features = Concatenate()([item_embedding, item_type_input, item_topics_input])

item_tower = Dense(128, activation='relu')(item_features)
item_tower = Dense(64, activation='relu')(item_tower)

# Combine the outputs of the two towers
output = Concatenate()([user_tower, item_tower])
output = Dense(32, activation='relu')(output)
output = Dense(1, activation='sigmoid')(output)

# Define the model
model = Model(inputs=[user_input, user_age_input, user_gender_input, user_country_input, item_input, item_type_input, item_topics_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# Note: you might need to adjust the input data format based on how you've preprocessed your data
model.fit([train['user_id'], train['age'], train['gender'], train['country'], train['item_id'], train['type'], train['Topics']], train['Timespent'], epochs=5, batch_size=32)

# Evaluate the model
model.evaluate([test['user_id'], test['age'], test['gender'], test['country'], test['item_id'], test['type'], test['Topics']], test['Timespent'])
